In [8]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras 
import SimpleITK as sitk
from keras.utils import normalize
from keras.metrics import MeanIoU

AttributeError: module 'keras.utils' has no attribute 'generic_utils'

In [6]:
# Specify the parameters
SIZE_X = 128 
SIZE_Y = 128
n_classes = 8

### Read train and mask files into numpy array

In [7]:
def read_volumetric_data():
    train_path = '../data/MM_WHS/ct_train'
    mask_path = '../data/MM_WHS/ct_train_mask'
    
    train_files = glob.glob(os.path.join(train_path, '*.nii.gz'))
    mask_files = glob.glob(os.path.join(mask_path, '*.nii.gz'))

    train_images = []
    for index_file in range(3):
        pat = sitk.ReadImage(train_files[index_file], sitk.sitkFloat32)
        pat = sitk.GetArrayFromImage(pat)
        for i in range(pat.shape[0]):
            train_images.append(pat[i])
            
    train_masks = []
    for index_file in range(3):
        pat = sitk.ReadImage(mask_files[index_file], sitk.sitkFloat32)
        pat = sitk.GetArrayFromImage(pat)
        for i in range(pat.shape[0]):
            train_masks.append(pat[i])
        
    train_images = np.array(train_images)
    train_masks = np.array(train_masks)
    return train_images, train_masks


train_images, train_masks = read_volumetric_data()
train_images.shape, train_masks.shape

((900, 512, 512), (900, 512, 512))

### Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)

train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)
np.unique(train_masks_encoded_original_shape)

### Train Test Split

In [ ]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.5, random_state = 0)

print("Class values in the dataset are ... ", np.unique(y_train))  

### To category

In [ ]:
from keras.utils import to_categorical

train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))
test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))